In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
MISSING = -9999

In [3]:
df_train = pd.read_csv("../data/interim/train.csv")
df_test = pd.read_csv("../data/interim/test.csv")

/Users/tingwei758/opt/anaconda3/envs/kaggle/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (3,4,6,10,12,18,24,35,56,57,65,85,86,87,88,89,98,99,100,105,124,125,126,127,128,130,131,133,166,171,172,192,193,194,195,204,205,206,207,228,229,230,240,256,257,258,259,260,261,262) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/tingwei758/opt/anaconda3/envs/kaggle/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (3,6,10,12,21,24,35,85,87,88,89,100,124,126,127,128,130,131,133,166,172,177,193,194,195,205,206,207,220,222,228,229,230,240,257,258,260,261,262) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
categorical = ['v17', 'v20', 'v25', 'v78', 'v154', 'v155', 'v161', 'cntry']

categorical2 = ['v70', 'v71', 'v72', 'v73', 'v102', 'v103','v158', 'v159', 'v160', 
                'v163', 'v164', 'v169', 'v170', 'v190', 'v191','v216', 'v231']

yes_no_cols = ['v6', 'v7', 'v8', 'v9', 'v10', 'v11', 'v14', 'v18', 'v21', 'v22', 'v23',
               'v24', 'v43', 'v53', 'v77', 'v105', 'v108', 'v152', 'v157', 'v162',
               'v165', 'v172', 'v173', 'v187', 'v188', 'v221', 'v241', 'v242', 'v243',
               'v248', 'v254', 'v256', 'v257']

education_encoding1 = ['v56', 'v58', 'v60', 'v62']
education_encoding2 = ['v57', 'v59', 'v61', 'v63']
occupation_encoding = ['v150', 'v151']

# to_drop = ['v124', 'v125', 'v126', 'v127', 'v128', 'v129', 'v130', 'v131']

In [5]:
df_train["test"] = 0
df_test["test"] = 1

In [6]:
df = pd.concat([df_train, df_test]).reset_index(drop=True)

In [7]:
df = df.set_index('id')

In [8]:
df.shape

(39325, 273)

In [9]:
df['v98'] = pd.to_numeric(df['v98'], 'coerce')

In [10]:
df['happiness_ratio_cntry'] = df['v98'] / df.groupby('cntry')["v98"].transform(np.mean)

In [11]:
# k = df.apply(pd.to_numeric, args=('coerce',)).count().sort_values()

In [12]:
# print(k.to_string())

In [13]:
# df = df.drop(columns=list(set(k[k < 200].index).difference(categorical)))

In [14]:
# df = df.drop(columns=to_drop)

In [15]:
df[yes_no_cols] = df[yes_no_cols].replace(to_replace=["2", 2], value=0)
df[yes_no_cols] = df[yes_no_cols].replace(to_replace=["1", 1], value=1)

In [16]:
for col in education_encoding1:
    df[col] = pd.to_numeric(df[col], 'coerce')
    df[col] = df[col].replace({55:np.nan})

for col in education_encoding2:
    df[col] = pd.to_numeric(df[col], 'coerce')
    df[col] = df[col].replace({5555:np.nan})

In [17]:
for col in occupation_encoding:
    df[col] = df[col].apply(lambda x: x[:2])

In [18]:
one_hot = categorical + categorical2 + occupation_encoding

In [19]:
numeric = [col for col in df.columns if col not in one_hot + ['id', 'satisfied', 'test']]

In [20]:
# for col in numeric:
#     numeric_col = pd.to_numeric(df[col], errors='coerce')
#     na_indicator = numeric_col.isna()
#     if sum(na_indicator) > 0:
#         df[col+'_ind'] = (~na_indicator).astype(int)
#         df[col+'_missing'] = np.where(na_indicator, df[col], np.nan)
#     df[col] = numeric_col

In [21]:
# missing = [col for col in df.columns if col.endswith('_missing')]

In [22]:
for col in one_hot:
    df[col] = pd.Categorical(df[col])
df_encoded = pd.get_dummies(df, columns=one_hot, drop_first=True)

In [23]:
df_encoded = df_encoded.apply(pd.to_numeric, args=('coerce',))

In [24]:
df_encoded = df_encoded.drop(columns=df_encoded.nunique()[df_encoded.nunique() <= 1].index)

In [25]:
df_encoded["percentage_answered"] = df_encoded.T.count()

In [26]:
df_encoded -= df_encoded.min()
df_encoded /= df_encoded.max()

In [27]:
df_encoded = df_encoded.fillna(MISSING)

In [28]:
# df_encoded = df_encoded.fillna(df_encoded.min())

In [29]:
df_encoded.shape

(39325, 1666)

In [30]:
from sklearn.ensemble import ExtraTreesClassifier

In [31]:
et = ExtraTreesClassifier(n_estimators=800, random_state=0)

In [32]:
X = df_encoded[df_encoded["test"] == 0].drop(columns=["satisfied","test"])

In [33]:
y = df_encoded[df_encoded["test"] == 0]["satisfied"]

In [34]:
et.fit(X, y)

ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=800,
                     n_jobs=None, oob_score=False, random_state=0, verbose=0,
                     warm_start=False)

In [35]:
feature_importance = pd.DataFrame(np.array([X.columns, et.feature_importances_]).T, 
                                  columns=['feature', 'importance']) \
                        .sort_values("importance", ascending=False) \
                        .reset_index(drop=True)

In [36]:
print(feature_importance.to_string())

                    feature   importance
0                       v98    0.0213138
1     happiness_ratio_cntry    0.0186819
2                      v253   0.00820881
3                       v74   0.00811074
4                       v99   0.00810596
5                       v79    0.0071152
6                      v101   0.00665635
7                      v180   0.00604759
8                      v224    0.0060107
9                       v82   0.00596917
10                     v241   0.00540776
11                     v243   0.00537331
12                     v178   0.00515821
13                     v227   0.00511477
14                   v190_2    0.0051132
15                      v80   0.00510699
16                     v129   0.00509839
17                      v81   0.00500733
18                     v225   0.00498955
19                     v234   0.00493122
20                     v134   0.00491416
21                     v131    0.0048685
22                     v179   0.00470976
23              

In [37]:
non_important_cols = feature_importance[300:]['feature']

In [38]:
df_encoded = df_encoded.drop(columns=non_important_cols)

In [39]:
df_encoded.shape

(39325, 302)

In [40]:
df_encoded.to_csv("../data/processed/data_encoded_stacking.csv", index=True)